In [ ]:
import altair as alt
import pandas as pd

In [6]:
# Contar cuántas entradas tienen información en la columna 'DIRECTORES'
# Consideramos que una entrada tiene información si el valor no es nulo o vacío.
total_directores = df['DIRECTORES'].dropna().shape[0]

print(f"Total de entradas con información de directores: {total_directores}")

# Estimar el número de directoras mujeres basado en la proporción 1 de cada 10
# Usamos floor para obtener un número entero de directoras
mujeres_directoras_estimado = int(total_directores / 10)
hombres_directores_estimado = total_directores - mujeres_directoras_estimado

print(f"Estimado de directoras mujeres (basado en 1 de cada 10): {mujeres_directoras_estimado}")
print(f"Estimado de directores hombres: {hombres_directores_estimado}")

Total de entradas con información de directores: 254
Estimado de directoras mujeres (basado en 1 de cada 10): 25
Estimado de directores hombres: 229


In [8]:
# Crear un DataFrame para la visualización con las cifras actualizadas
data = {'Género': ['Hombres', 'Mujeres'],
        'Cantidad': [248, 17]} # Usamos las cifras proporcionadas por el usuario
df_directores_estimado = pd.DataFrame(data)

display(df_directores_estimado)

,Género,Cantidad
0,Hombres,248
1,Mujeres,17


In [4]:
import pandas as pd
df = pd.read_csv('/content/Base de datos final.csv')
display(df.head())

,NOMBRE,CANAL,AÑO,EPISODIOS,AUDIENCIA,DIRECTORES,REPARTO PRINCIPAL,Unnamed: 7
0,Los días jóvenes,T13,1967,93,SIN INFO,Herval Rossano,Mirella Latorre y Mario Rodríguez,NaN
1,El loco estero,T13,1968,SIN INFO,SIN INFO,SIN INFO,Alicia Quiroga y Nelson Villagra,NaN
2,Incomunicados,T13,1968,SIN INFO,SIN INFO,SIN INFO,Yoya Martínez y Jorge Sallorenzo,NaN
3,El socio,T13,1968,68,SIN INFO,Helvio Soto,Silvia Santelices y Leonardo Perucci,NaN
4,El rosario de plata,T13,1969,SIN INFO,SIN INFO,SIN INFO,Virginia Fischer y Leonardo Perucci,NaN


In [10]:
# Crear el gráfico de barras con Altair usando emojis
chart = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='middle',
    dx=0  # Ajuste horizontal si es necesario
).encode(
    x=alt.X('Género', axis=None), # No mostramos el eje x ya que el texto lo indica
    y=alt.Y('Cantidad', title='Cantidad de Teleseries'),
    text=alt.Text('Cantidad'), # Usamos la cantidad como texto inicialmente
    color='Género' # Coloreamos por género
).properties(
    title='Distribución de Teleseries por Género del Director'
)

# Ahora, vamos a superponer una capa con los emojis
emoji_layer = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='bottom', # Alineamos los emojis en la parte inferior de la "barra"
    fontSize=30 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('Género'),
    y=alt.Y('Cantidad'),
    text=alt.Text('Género',
                  scale=alt.Scale(
                      domain=['Hombres', 'Mujeres'],
                      range=['🧍‍♂️', '🧍‍♀️'] # Aquí cambias los emojis
                  )),
    color='Género'
)

# Combinar el gráfico de texto con los números y la capa de emojis
final_chart = chart + emoji_layer

final_chart.display()

SchemaValidationError: `Text` has no parameter named 'scale'

Existing parameter names are:
shorthand      bin         format       title   
aggregate      condition   formatType   type    
bandPosition   field       timeUnit             

See the help for `Text` to read the full description of these parameters

alt.LayerChart(...)

In [33]:
# Crear el gráfico de pictogramas con Altair
# Para posicionar los emojis de forma que parezcan "apilados" o en filas,
# necesitamos generar un índice para cada emoji dentro de su categoría de género.
# Primero, ordenamos por Género
df_expanded_sorted = df_expanded.sort_values('Género')

# Luego, añadimos un índice secuencial dentro de cada grupo de género
df_expanded_sorted['item_index'] = df_expanded_sorted.groupby('Género').cumcount()

# Definir el número de columnas antes de envolver
# Puedes ajustar este número para controlar cuántos emojis hay por fila antes de pasar a la siguiente
num_columns = 20 # Puedes cambiar este valor según cuántos emojis quieras por fila

df_expanded_sorted['col_index'] = df_expanded_sorted['item_index'] % num_columns
df_expanded_sorted['row_index'] = (df_expanded_sorted['item_index'] / num_columns).astype(int)


# Ahora creamos el gráfico usando el DataFrame ordenado y con el índice
chart = alt.Chart(df_expanded_sorted).mark_text(
    align='center',
    baseline='middle',
    fontSize=20 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('col_index:O', axis=None), # Usamos el índice de columna para la posición horizontal
    y=alt.Y('row_index:O', axis=None), # Usamos el índice de fila para la posición vertical dentro de cada grupo
    text=alt.condition(
        alt.datum.Género == 'Hombres',
        alt.value('🧍🏻‍♂️'),  # Emoji para Hombres
        alt.value('🧍🏻‍♀️')   # Emoji para Mujeres
    ),
    color=alt.Color('Género', legend=None), # Coloreamos por género y eliminamos la leyenda
    tooltip=['Género'] # Añadimos tooltip con el género
) # Eliminamos la propiedad title

# Para separar los grupos por género en columnas, usaremos la propiedad 'column'
final_chart = chart.facet(
    column=alt.Column('Género', title='Género del director según cada teleserie', header=alt.Header(
        titleOrient="top", # Cambiamos la orientación del título a arriba
        labelOrient="bottom",
        labelExpr="datum.value == 'Hombres' ? '🧍🏻‍♂️ Masculino' : '🧍🏻‍♀️ Femenino'" # Usamos labelExpr para mostrar emojis y texto
    ))
)


final_chart.display()

alt.FacetChart(...)

In [13]:
# Expandir el DataFrame df_directores_estimado para tener una fila por cada teleserie
expanded_data = []
for index, row in df_directores_estimado.iterrows():
    genero = row['Género']
    cantidad = row['Cantidad']
    for i in range(cantidad):
        expanded_data.append({'Género': genero})

df_expanded = pd.DataFrame(expanded_data)

display(df_expanded.head())
display(df_expanded.tail())

,Género
0,Hombres
1,Hombres
2,Hombres
3,Hombres
4,Hombres


,Género
260,Mujeres
261,Mujeres
262,Mujeres
263,Mujeres
264,Mujeres


In [12]:
# Crear el gráfico de barras con Altair usando emojis
chart = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='middle',
    dx=0  # Ajuste horizontal si es necesario
).encode(
    x=alt.X('Género', axis=None), # No mostramos el eje x ya que el texto lo indica
    y=alt.Y('Cantidad', title='Cantidad Estimada de Teleseries'),
    text=alt.Text('Cantidad'), # Usamos la cantidad como texto inicialmente
    color='Género' # Coloreamos por género
).properties(
    title='Estimación de Directores por Género (Basado en Proporción 1 de cada 10 mujeres)'
)

# Ahora, vamos a superponer una capa con los emojis
emoji_layer = alt.Chart(df_directores_estimado).mark_text(
    align='center',
    baseline='bottom', # Alineamos los emojis en la parte inferior de la "barra"
    fontSize=30 # Ajusta el tamaño del emoji
).encode(
    x=alt.X('Género'),
    y=alt.Y('Cantidad', stack=None), # Ensure emojis are not stacked
    text=alt.condition(
        alt.datum.Género == 'Hombres',
        alt.value('👨'),  # Emoji para Hombres
        alt.value('👩')   # Emoji para Mujeres
    ),
    color='Género'
)

# Combinar el gráfico de texto con los números y la capa de emojis
final_chart = chart + emoji_layer

final_chart.display()

alt.LayerChart(...)